In [23]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime, timedelta
import random 

In [24]:
stock = 'SRFM'
t = 12

In [25]:
def plot_stock_candlestick(symbol, months=3):
    """
    Plot candlestick chart for any stock
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'TSLA')
        months: Number of months of historical data (default: 3)
    """
    # Get data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=months*30)
    
    df = yf.download(symbol, start=start_date, end=end_date, progress=False)
    ticker = yf.Ticker(symbol)
    company_name = ticker.info.get('longName', symbol)
    
    # Process data
    df.reset_index(inplace=True)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    
    # Create figure
    fig = go.Figure(data=[go.Candlestick(
        x=df['Date'],
        open=df['Open'].squeeze(),
        high=df['High'].squeeze(),
        low=df['Low'].squeeze(),
        close=df['Close'].squeeze()
    )])
    
    # Calculate stats
    current_price = df['Close'].iloc[-1]
    first_price = df['Close'].iloc[0]
    price_change = current_price - first_price
    percent_change = (price_change / first_price) * 100
    
    # Update layout
    fig.update_layout(
        title=f'{company_name} ({symbol}) - Last {months} Months<br>' +
              f'<sub>Current: ${current_price:.2f} | Change: ${price_change:.2f} ({percent_change:+.2f}%)</sub>',
        yaxis_title=f'{symbol} Stock Price (USD)',
        xaxis_title='Date',
        hovermode='x unified'
    )
    
    return fig

# ========== USE THE FUNCTION ==========
# Change the stock symbol and months here:
fig = plot_stock_candlestick(stock, months=t)
fig.show()

# You can easily plot multiple stocks:
# plot_stock_candlestick('TSLA', months=6).show()
# plot_stock_candlestick('MSFT', months=1).show()
# plot_stock_candlestick('GOOGL', months=12).show()

/var/folders/0h/ldts3ql12nldxhdrs9tccd_00000gn/T/ipykernel_53974/1616883206.py:13: FutureWarning:

YF.download() has changed argument auto_adjust default to True



In [26]:
# Get the same data for analysis
symbol = stock
months = t
end_date = datetime.now()
start_date = end_date - timedelta(days=months*30)

# Download data
df = yf.download(symbol, start=start_date, end=end_date, progress=False)

# Process data
df.reset_index(inplace=True)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)

# Convert column names to lowercase for the engulfing pattern function
df.columns = df.columns.str.lower()

print(f"Downloaded {len(df)} rows of {symbol} data")
df.tail()


Downloaded 246 rows of SRFM data


/var/folders/0h/ldts3ql12nldxhdrs9tccd_00000gn/T/ipykernel_53974/2252787733.py:8: FutureWarning:

YF.download() has changed argument auto_adjust default to True



Price,date,close,high,low,open,volume
241,2025-11-17,2.51,2.600,2.45,2.60,3069600
242,2025-11-18,2.38,2.555,2.36,2.42,2462700
243,2025-11-19,2.22,2.390,2.17,2.37,3073400
244,2025-11-20,2.06,2.530,2.06,2.35,4389500
245,2025-11-21,2.07,2.080,1.94,2.08,2760300


In [27]:
# Identify Bearish engulfing patterns

def Revsignal1(df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    signal = [0]*length
    bodydiff = [0]*length

    for row in range(1,length):
        bodydiff[row] = abs(open[row]-close[row])
        bodydiffmin = 0.003 
        
        if (bodydiff[row] > bodydiffmin and bodydiff[row-1] > bodydiffmin and 
            open[row-1] < close[row-1] and
            open[row] > close[row] and
            #open[row] >= close[row-1] and close[row] < open[row-1]:
            (open[row] - close[row]) >= +0e-5 and close[row] < open[row-1]): #Adding smudge factor to avoid equality issues
            signal[row] = 1 # Bearish engulfing
        elif (bodydiff[row] > bodydiffmin and bodydiff[row-1] > bodydiffmin and
            open[row-1] > close[row-1] and
            open[row] < close[row] and
            #open[row] <= close[row-1] and close[row] > open[row-1]:
            (open[row] - close[row-1]) <= -0e-5 and close[row] > open[row-1]): #Adding smudge factor to avoid equality issues
            signal[row] = 2 # Bullish engulfing 
        else:
            signal[row] = 0
        #signal[row] = random.choice([0,1,2])
    return signal

df['signal1'] = Revsignal1(df)
df[df['signal1']==1].count()


Price
date       35
close      35
high       35
low        35
open       35
volume     35
signal1    35
dtype: int64

In [28]:
# Identify Bullish engulfing patterns

def Revsignal1(df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    signal = [0]*length
    bodydiff = [0]*length

    for row in range(1,length):
        bodydiff[row] = abs(open[row]-close[row])
        bodydiffmin = 0.003 
        
        if (bodydiff[row] > bodydiffmin and bodydiff[row-1] > bodydiffmin and 
            open[row-1] < close[row-1] and
            open[row] > close[row] and
            #open[row] >= close[row-1] and close[row] < open[row-1]:
            (open[row] - close[row]) >= +0e-5 and close[row] < open[row-1]): #Adding smudge factor to avoid equality issues
            signal[row] = 1 # Bearish engulfing
        elif (bodydiff[row] > bodydiffmin and bodydiff[row-1] > bodydiffmin and
            open[row-1] > close[row-1] and
            open[row] < close[row] and
            #open[row] <= close[row-1] and close[row] > open[row-1]:
            (open[row] - close[row-1]) <= -0e-5 and close[row] > open[row-1]): #Adding smudge factor to avoid equality issues
            signal[row] = 2 # Bullish engulfing 
        else:
            signal[row] = 0
        #signal[row] = random.choice([0,1,2])
    return signal

df['signal1'] = Revsignal1(df)
df[df['signal1']==2].count()


Price
date       5
close      5
high       5
low        5
open       5
volume     5
signal1    5
dtype: int64

In [29]:
# Visualize engulfing patterns on candlestick chart

# Create candlestick chart with dynamic symbol name
fig = go.Figure(data=[go.Candlestick(
    x=df['date'],
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'],
    name=str(stock) if 'stock' in locals() else 'stock'
)])

# Add bearish engulfing markers (signal = 1)
bearish = df[df['signal1'] == 1]
fig.add_trace(go.Scatter(
    x=bearish['date'],
    y=bearish['high'] * 1.02,  # Position above the high
    mode='markers',
    marker=dict(
        symbol='triangle-down',
        size=12,
        color='red',
        line=dict(color='darkred', width=1)
    ),
    name='Bearish Engulfing',
    hovertemplate='<b>Bearish Engulfing</b><br>Date: %{x}<br>High: $%{y:.2f}<extra></extra>'
))

# Add bullish engulfing markers (signal = 2)
bullish = df[df['signal1'] == 2]
fig.add_trace(go.Scatter(
    x=bullish['date'],
    y=bullish['low'] * 0.98,  # Position below the low
    mode='markers',
    marker=dict(
        symbol='triangle-up',
        size=12,
        color='green',
        line=dict(color='darkgreen', width=1)
    ),
    name='Bullish Engulfing',
    hovertemplate='<b>Bullish Engulfing</b><br>Date: %{x}<br>Low: $%{y:.2f}<extra></extra>'
))

# Update layout
fig.update_layout(
    title=f'{stock} Candlestick Chart with Engulfing Patterns<br>' +
          f'<sub>Bearish: {len(bearish)} | Bullish: {len(bullish)}</sub>',
    yaxis_title='Stock Price (USD)',
    xaxis_title='Date',
    hovermode='x unified',
    height=600
)

fig.show()

print(f"Found {len(bearish)} bearish engulfing patterns")
print(f"Found {len(bullish)} bullish engulfing patterns")


Found 35 bearish engulfing patterns
Found 5 bullish engulfing patterns
